# Long Sentence Resolution - Fine-tune Splitting Model

# Documentation

## Connect Google Drive

*   Run the cell "Connect Google Drive" and on the popup window click  "Connect to Google Drive".

*   In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Modeling train data**: The modeling train data that will be loaded for fine-tuning the model.

* **Modeling validation data**: The modeling validation data that will be loaded for fine-tuning the model.

* **Modeling test data**: The modeling test data that will be loaded for fine-tuning the model.

* **Pre-trained model key**: Pre-trained model key (default: facebook/bart-large).

* **Pre-trained tokenizer key**: Pre-trained tokenizer key (default: facebook/bart-large).

* **RNG seed**: The seed that will be used for the random number generator when fine-tuning the model (default: 42).


## Login to Weights & Biases

* Run the cell "Login to Weights & Biases" 

* Find your API key in your profile here: https://wandb.ai/authorize

* Paste your API key from your profile to the textbox and hit enter

# Installations

In [ ]:
!pip install transformers==4.23.1 \
             datasets==2.5.2      \
             evaluate==0.3.0      \
             sacrebleu==2.3.1     \
             wandb==0.13.4        

# Downloads

## Hugging Face Transformers

In [ ]:
!wget -qO - https://github.com/huggingface/transformers/archive/refs/tags/v4.23.1.tar.gz | tar --strip-components 1 --one-top-level=transformers -xzf -

# Imports

In [ ]:
import os, datetime, json

import pandas as pd

from pathlib import Path

from typing import Union

from google.colab import drive

import ipywidgets as widgets

# Functions / Classes

In [ ]:
def convert_csv_file_to_translation_custom_json_lines(csv_file_path: Union[Path, str], json_lines_file_path: Union[Path, str], source_lang_column: str, target_lang_column: str):
  with open(json_lines_file_path, 'w', encoding='utf8') as json_lines_file:
    for json_line_dict in map(lambda o: { 'translation' : o }, pd.read_csv(csv_file_path, usecols=[source_lang_column, target_lang_column]).to_dict('records')):
      json_lines_file.write(json.dumps(json_line_dict, ensure_ascii=False) + '\n')

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
MODELING_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/modeling')

MODELS_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Models')

MODELS_DIRECTORY_PATH.mkdir(parents=True, exist_ok=True)

# Login to Weights & Biases

In [ ]:
!wandb login

# Configuration parameters

In [ ]:
modeling_train_data_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_dataset'), os.listdir(MODELING_DATA_DIRECTORY_PATH))))
modeling_validation_data_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_dataset'), os.listdir(MODELING_DATA_DIRECTORY_PATH))))
modeling_test_data_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('splitting_dataset'), os.listdir(MODELING_DATA_DIRECTORY_PATH))))

pre_trained_model_key_dropdown = widgets.Dropdown(options=['facebook/bart-large'])
pre_trained_tokenizer_key_dropdown = widgets.Dropdown(options=['facebook/bart-large'])
rng_seed_bounded_int_text = widgets.BoundedIntText(value=42, min=1,  max=10000, step=1)

display(widgets.HBox([widgets.Label(value="Modeling train data:"), modeling_train_data_dropdown]))
display(widgets.HBox([widgets.Label(value="Modeling validation data:"), modeling_validation_data_dropdown]))
display(widgets.HBox([widgets.Label(value="Modeling test data:"), modeling_test_data_dropdown]))

display(widgets.HBox([widgets.Label(value="Pre-trained model key:"), pre_trained_model_key_dropdown]))
display(widgets.HBox([widgets.Label(value="Pre-trained tokenizer key:"), pre_trained_tokenizer_key_dropdown]))
display(widgets.HBox([widgets.Label(value="RNG Seed:"), rng_seed_bounded_int_text]))

# Convert modeling data from CSV to JSON Lines format

In [ ]:
convert_csv_file_to_translation_custom_json_lines(csv_file_path=MODELING_DATA_DIRECTORY_PATH / modeling_train_data_dropdown.value / 'train.csv', json_lines_file_path=f'{modeling_train_data_dropdown.value}_train.json', source_lang_column='source', target_lang_column='target')

convert_csv_file_to_translation_custom_json_lines(csv_file_path=MODELING_DATA_DIRECTORY_PATH / modeling_validation_data_dropdown.value / 'validation.csv', json_lines_file_path=f'{modeling_validation_data_dropdown.value}_validation.json', source_lang_column='source', target_lang_column='target')

convert_csv_file_to_translation_custom_json_lines(csv_file_path=MODELING_DATA_DIRECTORY_PATH / modeling_test_data_dropdown.value / 'test.csv', json_lines_file_path=f'{modeling_test_data_dropdown.value}_test.json', source_lang_column='source', target_lang_column='target')

# Model Fine-tuning / Evaluation

In [ ]:
FINE_TUNED_MODEL_NAME = f'splitting_model_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}'

%env WANDB_PROJECT=long-sentence-resolution
%env WANDB_ENTITY=medoidai
%env WANDB_RUN_GROUP=splitting
%env WANDB_NAME=$FINE_TUNED_MODEL_NAME
%env PRE_TRAINED_MODEL_KEY=$pre_trained_model_key_dropdown.value
%env PRE_TRAINED_TOKENIZER_KEY=$pre_trained_tokenizer_key_dropdown.value
%env RNG_SEED=$rng_seed_bounded_int_text.value
%env TRAIN_FILE_PATH={modeling_train_data_dropdown.value}_train.json
%env VALIDATION_FILE_PATH={modeling_validation_data_dropdown.value}_validation.json
%env TEST_FILE_PATH={modeling_test_data_dropdown.value}_test.json
%env OUTPUT_DIRECTORY_PATH=$MODELS_DIRECTORY_PATH/$FINE_TUNED_MODEL_NAME

In [ ]:
!python transformers/examples/pytorch/translation/run_translation.py \
    --model_name_or_path "$PRE_TRAINED_MODEL_KEY" \
    --tokenizer "$PRE_TRAINED_TOKENIZER_KEY" \
    --learning_rate 5e-05 \
    --do_train \
    --do_eval \
    --do_predict \
    --num_train_epochs 3 \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --logging_strategy epoch \
    --seed "$RNG_SEED" \
    --train_file "$TRAIN_FILE_PATH" \
    --validation_file "$VALIDATION_FILE_PATH" \
    --test_file "$TEST_FILE_PATH" \
    --output_dir "$OUTPUT_DIRECTORY_PATH" \
    --source_lang source \
    --target_lang target \
    --logging_steps 10 \
    --per_device_train_batch_size=8 \
    --per_device_train_batch_size=8 \
    --predict_with_generate \
    --group_by_length \
    --load_best_model_at_end \
    --metric_for_best_model bleu \
    --report_to wandb \
    --run_name "$WANDB_NAME"